# Visualization of the coordinates of the location of apartments for rent and sale from previously downloaded data

In [1]:
!pip install ipython-sql
!pip install psycopg2-binary
!pip install pandas
!pip install numpy
!pip install numpy matplotlib
!pip install folium
!pip install geojson

In [2]:
import numpy as np
import pandas as pd
import psycopg2
import pandas.io.sql as sqlio
import matplotlib.pyplot as plt
import pylab as pl
import folium
import json
import os
from folium import plugins
%matplotlib inline

## Connecting to the database

In [3]:
psql = psycopg2.connect(host='192.168.10.163', port='5432', database='Otodom', user='barto', password='biznes')

### Retrieving the sale and rent table from the database

In [4]:
cur = psql.cursor()
sql_rent = "select * from Warsaw_rent;"
sql_sell = "select * from Warsaw_sell;"
dat_rent = sqlio.read_sql_query(sql_rent, psql)
dat_sell = sqlio.read_sql_query(sql_sell, psql)
conn = None

## Create a map

In [5]:
my_map = folium.Map()
world = folium.Map(
    zoom_start=11,
    location=[52.2380549, 21.0293513], prefer_canvas=True)
# prefer_canvas=True - loading map faster

Save the map

In [6]:
world.save('Map_rent+sell.html')

Selecting geographic coordinates from the sale and rent tables

In [7]:
lok_sell = dat_sell[["latitude","longitude"]]
lok_rent = dat_rent[["latitude","longitude"]]

Data cleaning

In [8]:

a = 0
for index, row in lok_sell.iterrows():
    if np.isnan(lok_sell.at[index,'latitude']) or np.isnan(lok_sell.at[index,'longitude']):
        lok_sell = lok_sell.drop([index])
        a += 1
print("Drop rows where Nan from table sell: ", a)

b = 0
for index, row in lok_rent.iterrows():
    if np.isnan(lok_rent.at[index,'latitude']) or np.isnan(lok_rent.at[index,'longitude']):
        lok_rent = lok_rent.drop([index])
        b += 1
print("Drop rows where Nan from table sell: ", b)

Drop rows where Nan from table sell:  155
Drop rows where Nan from table sell:  180


In [9]:
print(lok_rent.shape)
print(lok_sell.shape)

(2318, 2)
(2417, 2)


Index reset

In [10]:
lok_sell = lok_sell.reset_index(drop=True)
lok_rent = lok_rent.reset_index(drop=True)

In [11]:
world = plugins.MarkerCluster().add_to(world)

Add markers to the map - rent

In [12]:
for row in range(len(lok_rent.index)):
    folium.CircleMarker(
        location=[lok_rent.at[row,'latitude'], lok_rent.at[row,'longitude']],
        radius=3,
#         popup='My Birthplace',
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=1
    ).add_to(world)
    
    # '#428bca' - blue

Add markers to the map - sell

In [13]:
for row in range(len(lok_sell.index)):
    folium.CircleMarker(
        location=[lok_sell.at[row,'latitude'], lok_sell.at[row,'longitude']],
        radius=3,
#         popup='My Birthplace',
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=1
    ).add_to(world)

Save Map

In [14]:
world.save('Map_rent+sell.html')

Show Map

In [15]:
world